# Imports and Functions

In [ ]:
from datacube import Datacube
cdc = Datacube(config='/g/data/u46/users/ext547/ewater/cambodia_cube/cambodia.conf', app = "Polygon drill")
from datacube_stats.statistics import GeoMedian
from datacube.storage import masking
from datacube.storage.masking import mask_to_dict
from datacube.storage.storage import write_dataset_to_netcdf
from datacube.utils import geometry

import fiona
import rasterio.features
import geopandas as gpd

import numpy as np
import xarray as xr
import pickle

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation
import matplotlib.patheffects as PathEffects
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
%matplotlib inline

from skimage import exposure

import calendar

#Import external dea-notebooks functions using relative link to Scripts directory
import sys
import os.path
sys.path.append('/g/data/u46/users/sc0554/dea-notebooks/10_Scripts/')
import DEAPlotting

In [ ]:
#Define function to either read or load environmental data from Cambodia Cube
def LoadAreaOfInterest(study_area):
    '''
    Firstly, LoadAreaOfInterest checks whether a pickle, that contains an xarray of nbar data, 
    is saved in the output folder. If there is no pickle, the function searches the 
    "AreaOfInterest" function to gain lat/lon information for that particular study_area. If 
    there is no lat/lon coordinates in the AreaOfInterest function, then an error is returned. 
    If the coordinates are found, nbar data is loaded and masked. Finally data from all 
    sensors are concatenated together into one xarray.
   
    Last modified: March 2018
    
    Author: Erin Telfer
    
    Inputs: 
    study_area - the name of the study area
    '''
    
    study_area=study_area.lower().replace(" ","")
    lat_min, lat_max, lon_min, lon_max = AreaOfInterest(study_area)
            
    print("Loading Cambodia Cube data")
    dataset = {}

    #query is created
    query = {'time': (start_of_epoch, end_of_epoch),}
    query['x'] = (lon_min, lon_max)
    query['y'] = (lat_max, lat_min)
    query['crs'] = 'EPSG:4326'
    query['resolution'] = (-0.5,0.5)

    #print(query)

    for item in list_of_products: #loop through specified
        environmental_variable = cdc.load(product= item, group_by='solar_day', **query)
        #retrieve the projection information
        crs = environmental_variable.crs
        crswkt = environmental_variable.crs.wkt
        affine = environmental_variable.affine
        dataset[item] = environmental_variable

        print('loaded %s' % item) 
    print('ls load complete')


    #data from different sensors are joined together and sorted so that observations are sorted by time rather than sensor
    environmental_variable = xr.concat(dataset.values(), 'time')
    environmental_variable = environmental_variable.sortby('time')
    environmental_variable.attrs['crs'] = crs
    environmental_variable.attrs['affin|e'] = affine          
                    
    #print("saving data as {0}.pkl".format(study_area))

    #pickle.dump(nbar_clean, open(pickle_location,"wb")) #save nbar as pickle
    return environmental_variable

In [ ]:
#Define function to define the coordinates for the study area#Define 
def AreaOfInterest(study_area):
    if study_area == 'phumsrahkaev':
        lat_min = 13.000 #down
        lat_max = 13.100 #up
        lon_min = 103.300 #left
        lon_max = 103.400 #right  
    elif study_area == 'outapaong':
        lat_min = 12.600 #down
        lat_max = 12.800 #up
        lon_min = 103.600 #left
        lon_max = 103.800 #right
    elif study_area == 'mondulkiri':
        lat_min = 12.863 #down
        lat_max = 13.663 #up
        lon_min = 106.350 #left
        lon_max = 107.236 #right
    elif study_area == 'krongstungtreng':
        lat_min = 13.181 #down
        lat_max = 13.681 #up
        lon_min = 105.781 #left
        lon_max = 106.381 #right
    elif study_area == 'kaohnheaek':
        lat_min = 13.000 #down
        lat_max = 13.100 #up
        lon_min = 107.000 #left
        lon_max = 107.100 #right
    elif study_area == 'neakleoang':
        lat_min = 11.246 #down
        lat_max = 11.532 #up
        lon_min = 105.141 #left
        lon_max = 105.380 #right
    elif study_area == 'tonlesaplake':
        lat_min = 13.020 #down
        lat_max = 13.120 #up
        lon_min = 103.740 #left
        lon_max = 103.840 #right
    elif study_area == 'maximum_extent':
        lat_min = 9.25 #down
        lat_max = 15.25 #up
        lon_min = 101.75 #left
        lon_max = 108.25 #right
    elif study_area == 'kampongchhnang':
        lat_min = 12 #down
        lat_max = 12.25 #up
        lon_min = 104.75 #left
        lon_max = 105.0 #right        
    else:
        print('FileNotFoundError')
    return (lat_min, lat_max, lon_min, lon_max)

In [ ]:
def write_your_netcdf(data, dataset_name, filename, crs):

    """
    This function turns an xarray dataarray into a dataset so we can write it to netcdf. 
    It adds on a crs definition from the original array. data = your xarray dataset, dataset_name 
    is a string describing your variable
    
    Last modified: May 2018
    Author: Bex Dunn    
    """ 
   
    #turn array into dataset so we can write the netcdf
    if isinstance(data,xr.DataArray):
        dataset= data.to_dataset(name=dataset_name)
    elif isinstance(data,xr.Dataset):
        dataset = data
    else:
        print('your data might be the wrong type, it is: '+type(data))
    #grab our crs attributes to write a spatially-referenced netcdf
    dataset.attrs['crs'] = crs

    try:
        write_dataset_to_netcdf(dataset, filename)
    except RuntimeError as err:
        print("RuntimeError: {0}".format(err))    

In [ ]:
#Define function to create subplots of all scenes within an array as subplots
def one_band_image_subplots(ds, num_cols, figsize = [10,40], left  = 0.125, 
                              right = 0.9, bottom = 0.1, top = 0.9, 
                              wspace = 0.2, hspace = 0.4):
    '''
    one_band_image_subplots takes a dataset with one band and multiple time steps, 
    and plots them in image. 
    Last modified: March 2018
    Author: Mike Barnes
    Modified by: Claire Krause and Erin Telfer
    
    Inputs: 
    ds -   Dataset containing the bands to be plotted
    num_cols - number of columns for the subplot
    
    Optional:
    figsize - dimensions for the output figure
    left  - the space on the left side of the subplots of the figure
    right - the space on the right side of the subplots of the figure
    bottom - the space on the bottom of the subplots of the figure
    top - the space on the top of the subplots of the figure
    wspace - the amount of width reserved for blank space between subplots
    hspace - the amount of height reserved for white space between subplots
    '''
    # Find the number of rows/columns we need, based on the number of time steps in ds
    fig = plt.figure(figsize = figsize)
    timesteps = ds.time.size
    num_rows = int(np.ceil(timesteps/num_cols))
    fig, axes = plt.subplots(num_rows, num_cols, figsize = figsize)
    fig.subplots_adjust(left  = left, right = right, bottom = bottom, top = top, 
                        wspace = wspace, hspace = hspace)
    try: #loop through all scenes, prepare imagery and create subplots
        for i, ax in enumerate(fig.axes):
            image_ds = ds.rainfall.isel(time =i)
            ax.set_title(str(image_ds.time.values)[0:10])
            ax.imshow(image_ds, interpolation = 'nearest') #plot image as subplot
    except IndexError: #if there are an odd number of plots, this code will allow plotting of images
        fig.delaxes(ax)
        plt.draw() 

# Datacube query and pre-processing

In [ ]:
#define study area
study_area = 'maximum_extent' #name of study area
#study_area=study_area.lower().replace(" ","") #reformat to remove uppercase and spaces

#define temporal range ()
start_of_epoch = '1998-01-01'
end_of_epoch =  '2016-12-01'

#specify output folder
output_folder= '/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_input/'

**Load the PET and Rainfall data from datacube**

In [ ]:
# load the PET and Rainfall from the datacube
list_of_products = ['rainfall_grids_1998_2017']
rainfall=LoadAreaOfInterest(study_area)
list_of_products = ['pet_grids_1979_2016']
pet=LoadAreaOfInterest(study_area)
pet = pet.rename({'PET': 'pet'})

In [ ]:
pet = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_input/pet_cambodia_1998_2016_monthly.nc')
rainfall = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_input/rainfall_cambodia_1998_2016_monthly.nc')

### Mask datacube using shapefile of Cambodia

In [ ]:
shape_file = os.path.expanduser("/g/data/u46/users/sc0554/drought_indices_cambodia/shapefiles/cambodia_boundary/KHM_adm0.shp")
with fiona.open(shape_file) as shapes:
        crs = geometry.CRS(shapes.crs_wkt)
        first_geometry = next(iter(shapes))['geometry']
        geom = geometry.Geometry(first_geometry, crs=crs)

In [ ]:
rainfall_mask = rasterio.features.geometry_mask([geom.to_crs(rainfall.geobox.crs) for geoms in [geom]],
                                           out_shape=rainfall.geobox.shape,
                                           transform=rainfall.geobox.affine,
                                           all_touched=True,
                                           invert=True)
pet_mask = rasterio.features.geometry_mask([geom.to_crs(pet.geobox.crs) for geoms in [geom]],
                                           out_shape=pet.geobox.shape,
                                           transform=pet.geobox.affine,
                                           all_touched=True,
                                           invert=True)

rainfall_masked = rainfall.where(rainfall_mask)
pet_masked = pet.where(pet_mask)

### Calculate monthly totals
**Plot an example date for queried data**

In [ ]:
environmental_data = {'rainfall': rainfall, 'pet':pet}
monthly_environmental_data = {}
#resample to monthly averages and assign spatial and time units
for key, data in environmental_data.items():
    print (key)
    data[key].values[data[key].values == -9999.9] = np.nan
    #sum the total amount for the month
    data = data.resample(time = 'm').sum()
    data.attrs['crs'] = environmental_data[key].crs
    data.attrs['affin|e'] =environmental_data[key].attrs['affin|e']
    data.time.attrs['units'] = environmental_data[key].time.units #'seconds since 1970-01-01 00:00:00'
    data[key].attrs['units'] = 'millimeter'
    data = data.transpose('latitude', 'longitude', 'time')
    monthly_environmental_data[key] = data

environmental_data_masked = {'rainfall': rainfall_masked, 'pet':pet_masked}
monthly_environmental_data_masked = {}
#resample to monthly averages and assign spatial and time units
for key, data in environmental_data_masked.items():
    print (key)
    data[key].values[data[key].values == -9999.9] = np.nan
    #sum the total amount for the month
    data = data.resample(time = 'm').sum()
    data.attrs['crs'] = environmental_data_masked[key].crs
    data.attrs['affin|e'] =environmental_data_masked[key].attrs['affin|e']
    data.time.attrs['units'] = environmental_data_masked[key].time.units #'seconds since 1970-01-01 00:00:00'
    data[key].attrs['units'] = 'millimeter'
    data = data.transpose('latitude', 'longitude', 'time')
    monthly_environmental_data_masked[key] = data

In [ ]:
plt.subplot(121)
monthly_environmental_data['rainfall'].rainfall.isel(time = 10).plot()
plt.subplot(122)
# monthly_environmental_data_masked['rainfall'].rainfall.isel(time = 10).plot()
monthly_environmental_data['pet'].pet.isel(time = 10).plot()

In [ ]:
plt.subplot(121)
monthly_environmental_data_masked['rainfall'].rainfall.isel(time = 10).plot()
plt.subplot(122)
monthly_environmental_data_masked['pet'].pet.isel(time = 10).plot()


In [ ]:
monthly_environmental_data_masked['pet'].pet.isel(time = 10).plot()


In [ ]:
monthly_environmental_data_masked['pet'].pet.isel(time = 10).plot()


In [ ]:
fig = plt.figure(figsize=(11,7))
monthly_environmental_data['rainfall'].rainfall.mean(dim = ['latitude', 'longitude']).plot()

In [ ]:
fig = plt.figure(figsize=(11,7))
monthly_environmental_data_masked['rainfall'].rainfall.mean(dim = ['latitude', 'longitude']).plot()

In [ ]:
fig = plt.figure(figsize=(11,7))
monthly_environmental_data_masked['pet'].pet.mean(dim = ['latitude', 'longitude']).plot()

In [ ]:
fig = plt.figure(figsize=(11,7))
monthly_environmental_data['pet'].pet.mean(dim = ['latitude', 'longitude']).plot()

In [ ]:
plt.imshow(monthly_environmental_data['rainfall'].rainfall.isel(time=1))

### Save out input data as NETCDF

In [ ]:
for key, data in monthly_environmental_data.items():
    write_your_netcdf(data = data, dataset_name = key, filename = '/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_input/{0}_cambodia_1998_2016_monthly.nc'.format(key), crs = data.crs)

# Calculate SPEI using climate_indices module

In [ ]:
! /g/data/u46/users/sc0554/drought_indices_cambodia/call_climate_indices.sh --index spei --periodicity monthly --netcdf_precip /g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_input/rainfall_cambodia_1998_2016_monthly_masked.nc --var_name_precip rainfall --netcdf_pet /g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_input/pet_cambodia_1998_2016_monthly_masked.nc --output_file_base /g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/masked_2 --var_name_pet pet --scales 3 --calibration_start_year 1998 --calibration_end_year 2016

**Read in, merge and write out modelled drought index data**

In [ ]:
cambodia_spei_03_gamma = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_spei_gamma_03.nc')
cambodia_spei_03_gamma = cambodia_spei_03_gamma.transpose('time', 'latitude', 'longitude')
cambodia_spei_06_gamma = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_spei_gamma_06.nc')
cambodia_spei_06_gamma = cambodia_spei_06_gamma.transpose('time', 'latitude', 'longitude')
cambodia_spei_12_gamma = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_spei_gamma_12.nc')
cambodia_spei_12_gamma = cambodia_spei_12_gamma.transpose('time', 'latitude', 'longitude')
cambodia_spei_03_pearson = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_spei_pearson_03.nc')
cambodia_spei_03_pearson = cambodia_spei_03_pearson.transpose('time', 'latitude', 'longitude')
cambodia_spei_06_pearson = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_spei_pearson_06.nc')
cambodia_spei_06_pearson = cambodia_spei_06_pearson.transpose('time', 'latitude', 'longitude')
cambodia_spei_12_pearson = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_spei_pearson_12.nc')
cambodia_spei_12_pearson = cambodia_spei_12_pearson.transpose('time', 'latitude', 'longitude')

cambodia_spei = [cambodia_spei_03_gamma,cambodia_spei_06_gamma,cambodia_spei_12_gamma, cambodia_spei_03_pearson, cambodia_spei_06_pearson, cambodia_spei_12_pearson]
cambodia_spei = xr.merge(cambodia_spei)

In [ ]:
cambodia_masked_spei_03_gamma = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_masked_spei_gamma_03.nc')
cambodia_masked_spei_03_gamma = cambodia_masked_spei_03_gamma.transpose('time', 'latitude', 'longitude')
cambodia_masked_spei_06_gamma = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_masked_spei_gamma_06.nc')
cambodia_masked_spei_06_gamma = cambodia_masked_spei_06_gamma.transpose('time', 'latitude', 'longitude')
cambodia_masked_spei_12_gamma = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_masked_spei_gamma_12.nc')
cambodia_masked_spei_12_gamma = cambodia_masked_spei_12_gamma.transpose('time', 'latitude', 'longitude')
cambodia_masked_spei_03_pearson = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_masked_spei_pearson_03.nc')
cambodia_masked_spei_03_pearson = cambodia_masked_spei_03_pearson.transpose('time', 'latitude', 'longitude')
cambodia_masked_spei_06_pearson = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_masked_spei_pearson_06.nc')
cambodia_masked_spei_06_pearson = cambodia_masked_spei_06_pearson.transpose('time', 'latitude', 'longitude')
cambodia_masked_spei_12_pearson = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_masked_spei_pearson_12.nc')
cambodia_masked_spei_12_pearson = cambodia_masked_spei_12_pearson.transpose('time', 'latitude', 'longitude')

cambodia_masked_spei = [cambodia_masked_spei_03_gamma,cambodia_masked_spei_06_gamma,cambodia_masked_spei_12_gamma, cambodia_masked_spei_03_pearson, cambodia_masked_spei_06_pearson, cambodia_masked_spei_12_pearson]
cambodia_masked_spei = xr.merge(cambodia_masked_spei)

In [ ]:
# Save transformed and combined SPEI xarray as netcdf
cambodia_spei.to_netcdf(path = '/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_spei.nc', format = 'NETCDF4')
cambodia_masked_spei.to_netcdf(path = '/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_masked_spei.nc', format = 'NETCDF4')

# Calculate SPEI Quartiles

In [ ]:
# Open SPEI datasets

cambodia_spei = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_spei.nc')
cambodia_masked_spei = xr.open_dataset('/g/data/u46/users/sc0554/drought_indices_cambodia/climate_indices_output/cambodia_masked_spei.nc')

In [ ]:
cambodia_mean_spei_03 = cambodia_masked_spei.spei_gamma_03.mean(dim=('latitude', 'longitude'))
cambodia_mean_spei_06 = cambodia_masked_spei.spei_gamma_06.mean(dim=('latitude', 'longitude'))

In [ ]:
spei_quantiles = cambodia_mean_spei_03.quantile([0,0.25, 0.5, 0.75, 1], dim = ['time'], keep_attrs = True, )
spei_quantiles = spei_quantiles.values.tolist()
print(spei_quantiles)

In [ ]:
# Quartiles

spei_q1 = cambodia_mean_spei_03.where((cambodia_mean_spei_03 < spei_quantiles[1]) &
                                 (cambodia_mean_spei_03 >= spei_quantiles[0]), drop=True)
spei_q2 = cambodia_mean_spei_03.where((cambodia_mean_spei_03 < spei_quantiles[2]) &
                                 (cambodia_mean_spei_03 >= spei_quantiles[1]), drop=True)
spei_q3 = cambodia_mean_spei_03.where((cambodia_mean_spei_03 < spei_quantiles[3]) &
                                 (cambodia_mean_spei_03 >= spei_quantiles[2]), drop=True)
spei_q4 = cambodia_mean_spei_03.where((cambodia_mean_spei_03 <= spei_quantiles[4]) &
                                 (cambodia_mean_spei_03 >= spei_quantiles[3]), drop=True)

In [ ]:
# Hard boundaries

# spei_q1 = cambodia_mean_spei_03.where((cambodia_mean_spei_03 < -1) &
#                                  (cambodia_mean_spei_03 >= -2.199104), drop=True)
# spei_q2 = cambodia_mean_spei_03.where((cambodia_mean_spei_03 < 0) &
#                                  (cambodia_mean_spei_03 >= -1), drop=True)
# spei_q3 = cambodia_mean_spei_03.where((cambodia_mean_spei_03 < 1) &
#                                  (cambodia_mean_spei_03 >= 0), drop=True)
# spei_q4 = cambodia_mean_spei_03.where((cambodia_mean_spei_03 <= 2.050907) &
#                                  (cambodia_mean_spei_03 >= 1), drop=True)

In [ ]:
spei_q1.time.to_netcdf("spei_q1_dates_hard.nc")
spei_q2.time.to_netcdf("spei_q2_dates_hard.nc")
spei_q3.time.to_netcdf("spei_q3_dates_hard.nc")
spei_q4.time.to_netcdf("spei_q4_dates_hard.nc")